### Imports

In [28]:
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.metrics import f1_score, classification_report, confusion_matrix, precision_recall_curve, auc
import time


### Loading the datasets

In [30]:
import pandas as pd

# Load
train_df = pd.read_csv("X_train_processed.csv")
test_df = pd.read_csv("X_test_processed.csv")

# Split into features and labels
X_train = train_df.drop("No-show", axis=1).values
y_train = train_df["No-show"].values

X_test = test_df.drop("No-show", axis=1).values
y_test = test_df["No-show"].values

# Convert to torch tensors
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.float32).view(-1, 1)

X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test, dtype=torch.float32).view(-1, 1)


### Neural network with pytorch

In [32]:
class NeuralNet(nn.Module):
    def __init__(self, input_dim):
        super(NeuralNet, self).__init__()
        self.fc1 = nn.Linear(input_dim, 32)
        self.fc2 = nn.Linear(32, 16)
        self.output = nn.Linear(16, 1)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = self.sigmoid(self.output(x))
        return x


In [33]:
input_dim = X_train_tensor.shape[1]
model = NeuralNet(input_dim)

criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=0.01)


### Training

In [35]:
epochs = 1000
losses = []
start_time = time.time()

for epoch in range(epochs):
    model.train()
    optimizer.zero_grad()
    
    outputs = model(X_train_tensor)
    loss = criterion(outputs, y_train_tensor)
    loss.backward()
    optimizer.step()
    
    losses.append(loss.item())
    if epoch % 10 == 0:
        print(f"Epoch {epoch}, Loss: {loss.item():.4f}")

training_time = time.time() - start_time
print("Training Time:", training_time)


Epoch 0, Loss: 0.6719
Epoch 10, Loss: 0.5139
Epoch 20, Loss: 0.4798
Epoch 30, Loss: 0.4720
Epoch 40, Loss: 0.4683
Epoch 50, Loss: 0.4658
Epoch 60, Loss: 0.4639
Epoch 70, Loss: 0.4619
Epoch 80, Loss: 0.4600
Epoch 90, Loss: 0.4580
Epoch 100, Loss: 0.4565
Epoch 110, Loss: 0.4553
Epoch 120, Loss: 0.4544
Epoch 130, Loss: 0.4536
Epoch 140, Loss: 0.4530
Epoch 150, Loss: 0.4525
Epoch 160, Loss: 0.4521
Epoch 170, Loss: 0.4517
Epoch 180, Loss: 0.4514
Epoch 190, Loss: 0.4510
Epoch 200, Loss: 0.4505
Epoch 210, Loss: 0.4502
Epoch 220, Loss: 0.4499
Epoch 230, Loss: 0.4496
Epoch 240, Loss: 0.4495
Epoch 250, Loss: 0.4492
Epoch 260, Loss: 0.4490
Epoch 270, Loss: 0.4493
Epoch 280, Loss: 0.4488
Epoch 290, Loss: 0.4486
Epoch 300, Loss: 0.4485
Epoch 310, Loss: 0.4483
Epoch 320, Loss: 0.4482
Epoch 330, Loss: 0.4482
Epoch 340, Loss: 0.4480
Epoch 350, Loss: 0.4482
Epoch 360, Loss: 0.4479
Epoch 370, Loss: 0.4479
Epoch 380, Loss: 0.4478
Epoch 390, Loss: 0.4477
Epoch 400, Loss: 0.4479
Epoch 410, Loss: 0.4477
Epo

### test prediction evaluation 

In [37]:
model.eval()
with torch.no_grad():
    y_probs = model(X_test_tensor).numpy().flatten()
    predictions = (y_probs > 0.2).astype(int)

print("Test Accuracy:", (predictions == y_test).mean())
print("F1-Score:", f1_score(y_test, predictions))
print("Confusion Matrix:\n", confusion_matrix(y_test, predictions))

precision, recall, _ = precision_recall_curve(y_test, y_probs)
pr_auc = auc(recall, precision)
print("PR-AUC: ", pr_auc)

print("Classification Report:\n", classification_report(y_test, predictions))


Test Accuracy: 0.6018912315627545
F1-Score: 0.44101391271202595
Confusion Matrix:
 [[9832 7850]
 [ 949 3471]]
PR-AUC:  0.3535913370487738
Classification Report:
               precision    recall  f1-score   support

           0       0.91      0.56      0.69     17682
           1       0.31      0.79      0.44      4420

    accuracy                           0.60     22102
   macro avg       0.61      0.67      0.57     22102
weighted avg       0.79      0.60      0.64     22102

